In [1]:
import pandas as pd
import dtaidistance.dtw_ndim
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import dtaidistance
from dtaidistance import dtw_ndim
from dtaidistance import dtw_visualisation
from dtaidistance import dtw_ndim_visualisation
import numpy as np
import matplotlib.pyplot as plt
import time
from random import randint
from tqdm import tqdm_notebook
%matplotlib inline

In [2]:
def MDTW_L2_dist(v1, v2, window=None):
    return dtaidistance.dtw_ndim.distance(v1, v2, window=window)

def MDTW_L2_path(matr):
    return dtaidistance.dtw.best_path(matr)

def MDTW_L2_matr(v1, v2):
    return dtaidistance.dtw_ndim.warping_paths(v1, v2)

def MDTW_L2_visualisation(v1, v2, path):
    # another way to visualise
    # dtaidistance.dtw_ndim_visualisation.plot_warping(v1, v2, path)
    for i in range(v1.shape[1]):
        dtaidistance.dtw_visualisation.plot_warping(v1[:, i], v2[:, i], path)
        
#def prepare_data(data):
#    return data.reshape(data.shape[0], data.shape[1] * data.shape[2])

#def MDTW_L2_dist_1d(v1, v2):
#    return data.reshape()

In [3]:
def MDTW_DTW_dist(v1, v2, metric=dtaidistance.dtw.distance, window=None):
    n = len(v1)
    m = len(v2)
    dp = np.zeros((n + 1, m + 1))
    dp[0][0] = 0
    for i in range(1, n + 1):
        dp[i][0] = np.inf
    for i in range(1, m + 1):
        dp[0][i] = np.inf
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if window is None or abs(i - j) < window:
                dp[i][j] = metric(v1[i - 1], v2[j - 1]) ** 2 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])
            elif abs(i - j) == window:
                if i - j == window:
                    dp[i][j] = metric(v1[i - 1], v2[j - 1]) ** 2 + min(dp[i - 1][j], dp[i - 1][j - 1])
                else:
                    dp[i][j] = metric(v1[i - 1], v2[j - 1]) ** 2 + min(dp[i][j - 1], dp[i - 1][j - 1])
            else:
                dp[i][j] = np.inf
                
    return dp[n][m] ** 0.5

In [4]:
def MDTW_DTW_matr(v1, v2, metric=dtaidistance.dtw.distance, window=None):
    n = len(v1)
    m = len(v2)
    dp = np.zeros((n + 1, m + 1))
    dp[0][0] = 0
    for i in range(1, n + 1):
        dp[i][0] = np.inf
    for i in range(1, m + 1):
        dp[0][i] = np.inf
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if window is None or abs(i - j) < window:
                dp[i][j] = metric(v1[i - 1], v2[j - 1]) ** 2 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])
            elif abs(i - j) == window:
                if i - j == window:
                    dp[i][j] = metric(v1[i - 1], v2[j - 1]) ** 2 + min(dp[i - 1][j], dp[i - 1][j - 1])
                else:
                    dp[i][j] = metric(v1[i - 1], v2[j - 1]) ** 2 + min(dp[i][j - 1], dp[i - 1][j - 1])
            else:
                dp[i][j] = np.inf
    return dp ** 0.5
   
    
def MDTW_DTW_path(v1, v2, matr):
    time_path = dtaidistance.dtw.best_path(matr)
    signal_path = []
    for el in time_path:
        matr = MDTW_L2_matr(v1[el[0]], v2[el[1]])[1]
        signal_path.append(dtaidistance.dtw.best_path(matr))
    return (time_path, signal_path)


def MDTW_DTW_visualisation(v1, v2, path):
    time_path, signal_path = path
    for i in range(len(time_path)):
        print('1 time series signal ' + str(time_path[i][0]) + '\n' +
                  '2 time series signal ' + str(time_path[i][1]))
        dtaidistance.dtw_visualisation.plot_warping(v1[time_path[i][0]], v2[time_path[i][1]], signal_path[i])
        plt.show()

In [5]:
def prepare_data(data):
    return data.reshape(data.shape[0], data.shape[1] * data.shape[2])

    
def prepared_L2(left, right):
    #print(left.shape, right.shape)
    return MDTW_L2_dist(left.reshape(dim, left.shape[0] // dim),
                        right.reshape(dim, right.shape[0] // dim))


def prepared_DTW(left, right):
    return MDTW_DTW_dist(left.reshape(dim, left.shape[0] // dim),
                        right.reshape(dim, right.shape[0] // dim))

In [22]:
class Experiment:
    def __init__(self, data, labels, eps=1e-9):
        self.data = data
        self.labels = labels
        self.eps = 1e-9
    
    def calc_dist_experiment(self, idx1, idx2):
        print(idx1, idx2, MDTW_L2_dist(self.data[idx1], self.data[idx2]) ** 2)
        print(idx1, idx2, MDTW_DTW_dist(self.data[idx1], self.data[idx2]) ** 2)
        
    def description_dist(self):
        equals = 0
        less = 0
        greater = 0
        max_diff = 0
        mean_diff = 0
        max_idx1 = 0
        max_idx2 = 0
        for idx1 in tqdm_notebook(range(len(self.data))):
            for idx2 in tqdm_notebook(range(len(self.data))):
                dst1 = MDTW_L2_dist(self.data[idx1], self.data[idx2])
                dst2 = MDTW_DTW_dist(self.data[idx1], self.data[idx2])
                if dst1 == dst2 or abs(dst1 - dst2) < self.eps:
                    equals += 1
                elif dst1 < dst2:
                    less += 1
                else:
                    greater += 1
                if dst1 - dst2 > max_diff:
                    max_diff = dst1 - dst2
                    max_idx1 = idx1
                    max_idx2 = idx2
                mean_diff += dst1 - dst2
        mean_diff /= len(data) * len(data)
        print('Calculate distance experiment results:')
        print('L2 = DTW', equals)
        print('L2 < DTW', less)
        print('L2 > DTW', greater)
        print('Maximum difference between L2 and DTW:', max_diff)
        print('Indexes of maximum difference:', max_idx1, max_idx2)
        print('Mean value of difference:', mean_diff)
        
    def time_experiment(self, iter_count=100, cnt_signals=None, info=True):
        if cnt_signals is None:
            cnt_signals = self.data.shape[1]
        L2_time = {'min': np.inf, 'max': 0, 'mean': 0}
        DTW_time = {'min': np.inf, 'max': 0, 'mean': 0}
        for i in range(iter_count):
            idx1 = randint(0, len(self.data) - 1)
            idx2 = randint(0, len(self.data) - 1)
            begin = time.time()
            MDTW_L2_dist(self.data[idx1][:cnt_signals], self.data[idx2][:cnt_signals])
            end = time.time()
            L2_time['min'] = min(L2_time['min'], end - begin)
            L2_time['max'] = max(L2_time['max'], end - begin)
            L2_time['mean'] += (end - begin) / iter_count
            
            begin = time.time()
            MDTW_DTW_dist(self.data[idx1][:cnt_signals], self.data[idx2][:cnt_signals])
            end = time.time()
            DTW_time['min'] = min(DTW_time['min'], end - begin)
            DTW_time['max'] = max(DTW_time['max'], end - begin)
            DTW_time['mean'] += (end - begin) / iter_count
            
        if info:
            print('L2 time result:')
            print('min:', L2_time['min'])
            print('max:', L2_time['max'])
            print('mean:', L2_time['mean'])
        
            print('DTW time result:')
            print('min:', DTW_time['min'])
            print('max:', DTW_time['max'])
            print('mean:', DTW_time['mean'])
        return (L2_time, DTW_time)
    
    #def _prepare_data(self, data):
    #    return data.reshape(data.shape[0], data.shape[1] * data.shape[2])
    
    #def _prepared_L2(self, prepared_data):
    #    data = prepared_data.reshape(self.data.shape[0], self.data.shape[1], self.data.shape[2])
        
    
            
    def KNeighbourClassifier_test(self, window=None):
        data = prepare_data(self.data)
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(data, self.labels,
                                                                                   test_size=0.3)
        print(data, self.labels)
        print(self.X_train, self.X_test, self.y_train, self.y_test)
        
        model_L2 = KNeighborsClassifier(n_neighbors=1, metric=prepared_L2)
        model_L2.fit(self.X_train, self.y_train)
        predicted_L2 = model_L2.predict(self.X_test)
        accuracy_L2 = accuracy_score(self.y_test, predicted_L2)
            
        model_DTW = KNeighborsClassifier(n_neighbors=1, metric=prepared_DTW)
        model_DTW.fit(self.X_train, self.y_train)
        predicted_DTW = model_DTW.predict(self.X_test)
        accuracy_DTW = accuracy_score(self.y_test, predicted_DTW)
        
        print(predicted_L2)
        print(predicted_DTW)
        print(accuracy_L2, accuracy_DTW)
        
            
        return (accuracy_L2, accuracy_DTW)

In [23]:
ecog = pd.read_csv('../data/ECoG.csv', sep=',')

In [24]:
ecog.head()

,ECoG_time,ECoG_ch1,ECoG_ch2,ECoG_ch3,ECoG_ch4,ECoG_ch5,ECoG_ch6,ECoG_ch7,ECoG_ch8,ECoG_ch9,...,ECoG_ch119,ECoG_ch120,ECoG_ch121,ECoG_ch122,ECoG_ch123,ECoG_ch124,ECoG_ch125,ECoG_ch126,ECoG_ch127,ECoG_ch128
0,0.000,48,-60,-174,-226,-123,-308,94,75,-40,...,-57,174,-229,-84,-40,-292,-264,-206,-269,-286
1,0.001,40,-87,-208,-256,-115,-350,86,61,-66,...,-69,170,-229,-101,-42,-256,-265,-231,-258,-310
2,0.002,46,-113,-210,-248,-124,-365,97,68,-96,...,-85,139,-213,-102,-11,-230,-248,-230,-217,-301
3,0.003,78,-112,-181,-220,-116,-368,115,104,-90,...,-98,103,-213,-127,11,-254,-253,-224,-210,-299
4,0.004,109,-115,-178,-216,-128,-342,111,123,-90,...,-117,70,-218,-152,30,-249,-238,-217,-243,-274


In [25]:
ecog.tail()

,ECoG_time,ECoG_ch1,ECoG_ch2,ECoG_ch3,ECoG_ch4,ECoG_ch5,ECoG_ch6,ECoG_ch7,ECoG_ch8,ECoG_ch9,...,ECoG_ch119,ECoG_ch120,ECoG_ch121,ECoG_ch122,ECoG_ch123,ECoG_ch124,ECoG_ch125,ECoG_ch126,ECoG_ch127,ECoG_ch128
663265,663.265,44,47,-41,-111,29,121,316,-828,116,...,240,156,-148,-360,-156,-215,67,-142,-2,-310
663266,663.266,78,46,-60,-60,50,140,345,-814,121,...,249,170,-141,-364,-142,-197,38,-132,-3,-373
663267,663.267,67,-1,-103,-29,122,188,375,-774,122,...,249,182,-155,-367,-135,-185,46,-113,-20,-395
663268,663.268,93,-34,-163,-62,186,261,426,-737,135,...,244,176,-175,-387,-145,-205,16,-132,-41,-438
663269,663.269,158,1,-142,-35,228,311,462,-709,165,...,232,166,-192,-372,-168,-239,-18,-199,-146,-495


In [26]:
print(ecog.shape)

(663270, 129)


In [27]:
event = pd.read_csv('../data/Event.csv', sep=',')

In [28]:
event.head()

,EventTime,EventData,EventIndex
0,0.000,35,0
1,0.001,40,1
2,0.002,36,2
3,0.003,34,3
4,0.004,35,4


In [29]:
print(event.shape)

(663270, 3)


In [30]:
prepared = event.EventData <= 100
iti = event[event.EventData.between(300, 400, inclusive=True)] #0
stimulus1 = event[event.EventData.between(650, 750, inclusive=True)] #45
stimulus2 = event[event.EventData.between(950, 1050, inclusive=True)] #90
stimulus3 = event[event.EventData.between(1300, 1400, inclusive=True)] #135
stimulus4 = event[event.EventData.between(1600, 1700, inclusive=True)] #180
stimulus5 = event[event.EventData.between(1950, 2050, inclusive=True)] #225
stimulus6 = event[event.EventData.between(2250, 2350, inclusive=True)] #270
stimulus7 = event[event.EventData.between(2600, 2700, inclusive=True)] #315
stimulus8 = event[event.EventData.between(2900, 3000, inclusive=True)] #360

In [31]:
stimulus1

,EventTime,EventData,EventIndex
82015,82.015,686,82015
82016,82.016,692,82016
82017,82.017,684,82017
82018,82.018,685,82018
82019,82.019,691,82019
...,...,...,...
645255,645.255,691,645255
645256,645.256,688,645256
645257,645.257,687,645257
645258,645.258,686,645258


[   0- 100] : preparation time before experiment
      [ 300- 400] : ITI
      [ 650- 750] : Stimulus 1 ( 45 degree)
      [ 950-1050] : Stimulus 2 ( 90 degree)
      [1300-1400] : Stimulus 3 (135 degree)
      [1600-1700] : Stimulus 4 (180 degree)
      [1950-2050] : Stimulus 5 (225 degree)
      [2250-2350] : Stimulus 6 (270 degree)
      [2600-2700] : Stimulus 7 (315 degree)
      [2900-3000] : Stimulus 8 (360 degree)

In [32]:
ts1 = ecog.iloc[stimulus1.EventIndex].to_numpy()[:, 1:]
ts2 = ecog.iloc[stimulus2.EventIndex].to_numpy()[:, 1:]
ts3 = ecog.iloc[stimulus3.EventIndex].to_numpy()[:, 1:]
ts4 = ecog.iloc[stimulus4.EventIndex].to_numpy()[:, 1:]
ts5 = ecog.iloc[stimulus5.EventIndex].to_numpy()[:, 1:]
ts6 = ecog.iloc[stimulus6.EventIndex].to_numpy()[:, 1:]
ts7 = ecog.iloc[stimulus7.EventIndex].to_numpy()[:, 1:]
ts8 = ecog.iloc[stimulus8.EventIndex].to_numpy()[:, 1:]

In [33]:
print(ts1.shape, ts2.shape)

(40072, 128) (40119, 128)


In [34]:
print(ts1)

[[ -36.  188.  324. ...  326.  -38.   23.]
 [ -38.  192.  314. ...  318.   -6.   83.]
 [ -48.  193.  298. ...  310.    9.  133.]
 ...
 [-245.  120.  378. ... -182.   -8.  -91.]
 [-233.   55.  321. ... -239.  -77. -121.]
 [-228.   69.  358. ... -234.  -66.  -79.]]


In [35]:
dim = 10

In [36]:
def experiment(data_class1, data_class2, sample_size=30, cnt_signals=30, dim=10):
    data_class1 = data_class1[:, :dim]
    data_class2 = data_class2[:, :dim]
    time_series1 = []
    time_series2 = []
    
    for i in range(sample_size):
        time_series1.append(data_class1[i + sample_size * np.arange(cnt_signals)])
        time_series2.append(data_class2[i + sample_size * np.arange(cnt_signals)])
    time_series1 = np.array(time_series1)
    time_series2 = np.array(time_series2)
    labels = []
    data = []

    for i in range(sample_size):
        data.append(time_series1[i])
        data.append(time_series2[i])
        labels.append(0)
        labels.append(1)
    data = np.array(data)
    labels = np.array(labels)
        
    exp = Experiment(data, labels)
    return exp.KNeighbourClassifier_test()

In [37]:
experiment(ts1, ts2)

[[ -36.  188.  324. ...  -18.  -71.  161.]
 [ -64. -143.   64. ...  -22.  108.  116.]
 [ -38.  192.  314. ...  -20.  -60.  186.]
 ...
 [ 145.  -75. -349. ... -236. -328. -103.]
 [ -19.   75.  -14. ... -106.   -1.  -31.]
 [ 209.   20. -272. ... -250. -394. -264.]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[ -34. -100.   64. ...  -45.  122.  190.]
 [  47.  342.  310. ... -174. -413. -196.]
 [ -71.  233.  298. ...  -99.  -98.  126.]
 ...
 [ -48.  291.  265. ... -171. -166.   27.]
 [-115.   68.  -22. ...   77. -148.   28.]
 [  38.  308.  281. ... -133. -353. -172.]] [[ -80.  132.   15. ...   39. -196.  -18.]
 [ 108.  -34.   41. ...  -21.  -26.  287.]
 [ 114.  -92. -291. ...  -77.  -59.  241.]
 ...
 [ -30.  -44.  122. ...  -77.   89.  196.]
 [  10.  390.  344. ... -250. -380. -137.]
 [ 130.   11.   24. ... -129. -170.  216.]] [1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 1 0 1 0 1 0 0 1 1 1 1 1 1 1 1 0
 0 1 0 0 0

(1.0, 1.0)

In [38]:
experiment(ts1, ts3)

[[-3.600e+01  1.880e+02  3.240e+02 ... -1.800e+01 -7.100e+01  1.610e+02]
 [ 2.410e+02  1.770e+02  1.100e+02 ... -3.199e+03 -1.460e+02 -3.270e+02]
 [-3.800e+01  1.920e+02  3.140e+02 ... -2.000e+01 -6.000e+01  1.860e+02]
 ...
 [ 1.700e+02  3.400e+01 -8.000e+00 ... -2.844e+03  8.500e+01  2.100e+02]
 [-1.900e+01  7.500e+01 -1.400e+01 ... -1.060e+02 -1.000e+00 -3.100e+01]
 [ 1.690e+02  6.000e+01 -1.600e+01 ... -2.905e+03  6.700e+01  1.800e+02]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[  321.    87.   174. ... -2794.    -7.  -122.]
 [  378.   148.   252. ... -2736.    72.   -41.]
 [  148.    13.    -5. ... -2847.    -3.  -174.]
 ...
 [  169.    60.   -16. ... -2905.    67.   180.]
 [  177.    18.     3. ... -2825.   -69.  -211.]
 [  383.   183.   177. ... -2740.    82.   -15.]] [[  -53.   187.   100. ...    -5.  -229.   -48.]
 [  -20.   352.   298. ...  -239.  -295.   -79.]
 [  219.   148.    60. ... -3089.  

(1.0, 1.0)

In [39]:
experiment(ts2, ts3)

[[  -64.  -143.    64. ...   -22.   108.   116.]
 [  241.   177.   110. ... -3199.  -146.  -327.]
 [  -34.  -100.    64. ...   -45.   122.   190.]
 ...
 [  170.    34.    -8. ... -2844.    85.   210.]
 [  209.    20.  -272. ...  -250.  -394.  -264.]
 [  169.    60.   -16. ... -2905.    67.   180.]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[   68.   -34.    60. ...    24.    10.   281.]
 [  173.    31.    41. ... -2805.    89.   164.]
 [   79.   -14.    65. ...   -98.  -126.   238.]
 ...
 [    9.    -5.   113. ...   -43.    66.   205.]
 [  108.   -34.    41. ...   -21.   -26.   287.]
 [  128.    -6.  -156. ...   -72.  -111.   233.]] [[  321.    87.   174. ... -2794.    -7.  -122.]
 [  149.     4.   -10. ... -2871.   -66.  -213.]
 [  143.    19.   -20. ... -2792.    78.  -134.]
 ...
 [    8.  -223.  -371. ...  -142.  -101.   210.]
 [   93.  -162.  -425. ...  -222.  -278.    23.]
 [  368.   198.   194. ... 

(1.0, 1.0)

In [40]:
experiment(ts1, ts4)

[[ -36.  188.  324. ...  -18.  -71.  161.]
 [  81.   92.   86. ...  153.   71.   69.]
 [ -38.  192.  314. ...  -20.  -60.  186.]
 ...
 [ 114.   82.   50. ...  -26. -122.   58.]
 [ -19.   75.  -14. ... -106.   -1.  -31.]
 [  77.   35.   29. ...   46.  -75.   67.]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[ 199.  141.   44. ... -153. -262.    2.]
 [  31.   33.  -32. ...  102.   76.  204.]
 [ 108.  116.  123. ...  178.   59.   91.]
 ...
 [ -17.    5.  -55. ... -127.  -49.  197.]
 [ -12.  259.  215. ...  -64. -285. -110.]
 [ -19.   75.  -14. ... -106.   -1.  -31.]] [[ 128.  114.   36. ... -233. -208.   91.]
 [ -17.  -14.  -95. ...    4.   35.  274.]
 [ -20.  352.  298. ... -239. -295.  -79.]
 ...
 [  77.   35.   29. ...   46.  -75.   67.]
 [  81.   92.   86. ...  153.   71.   69.]
 [ -72.  205.  306. ...  -69.  -79.  173.]] [1 1 1 1 1 0 0 0 1 0 1 1 0 0 0 0 1 0 0 1 1 1 1 0 1 0 1 1 1 1 0 0 0 0 0 0 1
 1 1 1 0 0

(1.0, 1.0)

In [41]:
experiment(ts2, ts4)

[[ -64. -143.   64. ...  -22.  108.  116.]
 [  81.   92.   86. ...  153.   71.   69.]
 [ -34. -100.   64. ...  -45.  122.  190.]
 ...
 [ 114.   82.   50. ...  -26. -122.   58.]
 [ 209.   20. -272. ... -250. -394. -264.]
 [  77.   35.   29. ...   46.  -75.   67.]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[ 126.   97.   48. ...  -89. -160.   27.]
 [  59.   65.   -2. ... -166. -151.  154.]
 [ 165.  122.   35. ... -154. -236.   -8.]
 ...
 [ 155.   34.  -49. ... -134. -163.  248.]
 [  35.   62.   21. ...  128.   68.  173.]
 [ -37.  -25. -122. ...  -29.   31.  274.]] [[ 159.  131.   70. ... -217. -233.   72.]
 [ 227.   20. -181. ... -108. -108.  220.]
 [ 103. -175. -446. ... -220. -244.   76.]
 ...
 [ 140.  123.   50. ... -165. -226.  -11.]
 [ 145.  -75. -349. ... -236. -328. -103.]
 [ 130.   11.   24. ... -129. -170.  216.]] [1 1 1 1 1 1 0 0 1 0 0 1 0 1 1 1 1 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 1 1 0
 0 1 0 1 1

(1.0, 1.0)

In [42]:
experiment(ts3, ts4)

[[  241.   177.   110. ... -3199.  -146.  -327.]
 [   81.    92.    86. ...   153.    71.    69.]
 [  219.   148.    60. ... -3089.   -61.  -255.]
 ...
 [  114.    82.    50. ...   -26.  -122.    58.]
 [  169.    60.   -16. ... -2905.    67.   180.]
 [   77.    35.    29. ...    46.   -75.    67.]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[  364.   174.   181. ... -2730.   102.    22.]
 [  165.   122.    35. ...  -154.  -236.    -8.]
 [  225.    38.    25. ... -2852.   -66.  -187.]
 ...
 [  390.   222.   228. ... -2764.    58.   -71.]
 [   81.    92.    86. ...   153.    71.    69.]
 [  399.   212.   301. ... -2782.    30.  -107.]] [[ 1.990e+02  1.410e+02  4.400e+01 ... -1.530e+02 -2.620e+02  2.000e+00]
 [-1.000e+01 -6.000e+00 -7.400e+01 ...  5.900e+01  4.500e+01  2.580e+02]
 [-1.700e+01 -1.400e+01 -9.500e+01 ...  4.000e+00  3.500e+01  2.740e+02]
 ...
 [ 1.280e+02  1.140e+02  3.600e+01 ... -2.330e+02 -2.

(1.0, 1.0)

In [43]:
experiment(ts1, ts5)

[[ -36.  188.  324. ...  -18.  -71.  161.]
 [-323.  -55.  -79. ...  -93.  -55.  164.]
 [ -38.  192.  314. ...  -20.  -60.  186.]
 ...
 [ 164.  102.  148. ...  208.  143. -249.]
 [ -19.   75.  -14. ... -106.   -1.  -31.]
 [  35.  -15.   66. ...  227.  197. -201.]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[  35.  -15.   66. ...  227.  197. -201.]
 [-333.    9.   30. ...  -66. -218.   66.]
 [-115.   68.  -22. ...   77. -148.   28.]
 ...
 [-286. -189.  -24. ...  287.    6.  -73.]
 [  83.  394.  332. ... -230. -462. -202.]
 [ -70.   72.   36. ...  -40.   68.   58.]] [[ -44.    2.  132. ...  373.   33. -133.]
 [  85.  130.  151. ...  187.  -91. -322.]
 [ -26.  325.  276. ... -208. -218.  -22.]
 ...
 [ -53.  187.  100. ...   -5. -229.  -48.]
 [-315.  -12. -120. ...   88. -242.  -60.]
 [  77.  112.   87. ...  166.   -4. -359.]] [1 1 0 1 0 1 1 1 1 1 0 0 0 0 0 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 0 0 1 1 0 1 1
 0 0 1 0 0

(1.0, 1.0)

In [44]:
experiment(ts2, ts5)

[[ -64. -143.   64. ...  -22.  108.  116.]
 [-323.  -55.  -79. ...  -93.  -55.  164.]
 [ -34. -100.   64. ...  -45.  122.  190.]
 ...
 [ 164.  102.  148. ...  208.  143. -249.]
 [ 209.   20. -272. ... -250. -394. -264.]
 [  35.  -15.   66. ...  227.  197. -201.]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[  79.  -14.   65. ...  -98. -126.  238.]
 [ 145.  -75. -349. ... -236. -328. -103.]
 [ 114.  -92. -291. ...  -77.  -59.  241.]
 ...
 [  77.  112.   87. ...  166.   -4. -359.]
 [ -44.    2.  132. ...  373.   33. -133.]
 [ 128.   -6. -156. ...  -72. -111.  233.]] [[  61.  139.  235. ...  348.   11. -177.]
 [  85.  109.  133. ...  137.  -97. -372.]
 [-288. -114.   -4. ...  279.  -44.  -56.]
 ...
 [  80.  105.  102. ...  104.  -61. -409.]
 [  85.  130.  151. ...  187.  -91. -322.]
 [-352.  -77. -135. ...  127. -209.  -78.]] [0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 1 1 1
 1 1 1 1 0

(1.0, 1.0)

In [45]:
experiment(ts3, ts5)

[[  241.   177.   110. ... -3199.  -146.  -327.]
 [ -323.   -55.   -79. ...   -93.   -55.   164.]
 [  219.   148.    60. ... -3089.   -61.  -255.]
 ...
 [  164.   102.   148. ...   208.   143.  -249.]
 [  169.    60.   -16. ... -2905.    67.   180.]
 [   35.   -15.    66. ...   227.   197.  -201.]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[   20.    -4.     4. ...   197.    54.  -304.]
 [ -333.     9.    30. ...   -66.  -218.    66.]
 [ -276.   -41.     0. ...   228.  -111.   -70.]
 ...
 [ -276.    62.    42. ...   -67.  -179.   111.]
 [  177.    18.     3. ... -2825.   -69.  -211.]
 [   77.   112.    87. ...   166.    -4.  -359.]] [[  326.   129.   133. ... -2710.   177.   184.]
 [  166.    46.   -33. ... -2900.    28.  -191.]
 [ -352.   -77.  -135. ...   127.  -209.   -78.]
 ...
 [  164.   102.   148. ...   208.   143.  -249.]
 [  168.    28.     4. ... -2839.   120.   234.]
 [  174.    26.    30. ... 

(1.0, 1.0)

In [46]:
experiment(ts4, ts5)

[[  81.   92.   86. ...  153.   71.   69.]
 [-323.  -55.  -79. ...  -93.  -55.  164.]
 [ 108.  116.  123. ...  178.   59.   91.]
 ...
 [ 164.  102.  148. ...  208.  143. -249.]
 [  77.   35.   29. ...   46.  -75.   67.]
 [  35.  -15.   66. ...  227.  197. -201.]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[  70.   30.   70. ...  203.   94. -272.]
 [-323.  -55.  -79. ...  -93.  -55.  164.]
 [ -10.   -6.  -74. ...   59.   45.  258.]
 ...
 [ 140.  123.   50. ... -165. -226.  -11.]
 [  85.  130.  151. ...  187.  -91. -322.]
 [-269. -138.   15. ...  303.  -17.  -51.]] [[-246.   68.   92. ...  -64. -266.  -22.]
 [ -19.   -2. -101. ...  -69.    9.  243.]
 [  61.  139.  235. ...  348.   11. -177.]
 ...
 [-352.  -77. -135. ...  127. -209.  -78.]
 [  20.   -4.    4. ...  197.   54. -304.]
 [ 159.  131.   70. ... -217. -233.   72.]] [1 1 0 1 1 0 1 1 0 0 0 1 0 1 1 0 1 0 1 0 1 0 0 0 0 0 1 0 1 1 1 0 1 0 1 1 0
 0 0 0 1 1

(1.0, 1.0)

In [47]:
experiment(ts1, ts6)

[[ -64. -143.   64. ...  -22.  108.  116.]
 [-339.   82.   71. ...   71.   32.   30.]
 [ -34. -100.   64. ...  -45.  122.  190.]
 ...
 [ 168.  -50.  -70. ... -263.   -1.  -66.]
 [ 209.   20. -272. ... -250. -394. -264.]
 [ 209.  -21.  -72. ... -305.  -75. -135.]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[  79.  -14.   65. ...  -98. -126.  238.]
 [ 103. -175. -446. ... -220. -244.   76.]
 [ -57.  -62.  -21. ...   78.   37.  269.]
 ...
 [  66.  -83.  -45. ...  -84.  261.  164.]
 [ 130.   11.   24. ... -129. -170.  216.]
 [-139.   17.  -15. ...   11.  543.  367.]] [[-414.  103.   91. ...   47.  122.   70.]
 [-339.   82.   71. ...   71.   32.   30.]
 [-124.  -11.  -22. ...   11.  553.  367.]
 ...
 [-393.   72.   59. ...   72.   95.   45.]
 [  91. -194. -412. ... -209. -184.  134.]
 [   9.   -5.  113. ...  -43.   66.  205.]] [0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 1 1 0 0 1 1 1 1 1 1 1 0 1 0 0 1 0 0 1 0 0 0
 1 1 1 0 1

(1.0, 1.0)

In [48]:
experiment(ts2, ts6)

[[ -64. -143.   64. ...  -22.  108.  116.]
 [-339.   82.   71. ...   71.   32.   30.]
 [ -34. -100.   64. ...  -45.  122.  190.]
 ...
 [ 168.  -50.  -70. ... -263.   -1.  -66.]
 [ 209.   20. -272. ... -250. -394. -264.]
 [ 209.  -21.  -72. ... -305.  -75. -135.]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[-170.  104.   81. ... -112.  407.  265.]
 [  93. -162. -425. ... -222. -278.   23.]
 [-361.  146.  140. ...  -61.  115.   71.]
 ...
 [-392.   97.  102. ...   68.  104.   69.]
 [ 209.   99. -134. ...  -96. -113.  297.]
 [   8. -223. -371. ... -142. -101.  210.]] [[ 155.   34.  -49. ... -134. -163.  248.]
 [ 130.   11.   24. ... -129. -170.  216.]
 [-124.  -11.  -22. ...   11.  553.  367.]
 ...
 [  38.  -73.  -43. ...  -40.  332.  199.]
 [ -49.  -92.  -39. ...   81.   36.  277.]
 [ 145.  -75. -349. ... -236. -328. -103.]] [1 0 1 0 0 1 0 0 1 0 0 1 0 1 1 0 0 0 1 0 0 1 1 1 0 0 1 1 0 1 1 0 0 0 0 1 1
 1 0 1 0 0

(1.0, 1.0)

In [49]:
experiment(ts3, ts6)

[[ 2.410e+02  1.770e+02  1.100e+02 ... -3.199e+03 -1.460e+02 -3.270e+02]
 [-3.390e+02  8.200e+01  7.100e+01 ...  7.100e+01  3.200e+01  3.000e+01]
 [ 2.190e+02  1.480e+02  6.000e+01 ... -3.089e+03 -6.100e+01 -2.550e+02]
 ...
 [ 1.680e+02 -5.000e+01 -7.000e+01 ... -2.630e+02 -1.000e+00 -6.600e+01]
 [ 1.690e+02  6.000e+01 -1.600e+01 ... -2.905e+03  6.700e+01  1.800e+02]
 [ 2.090e+02 -2.100e+01 -7.200e+01 ... -3.050e+02 -7.500e+01 -1.350e+02]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[  202.   118.    15. ... -2987.    20.  -187.]
 [  263.   100.   114. ... -2641.   236.   248.]
 [ -139.    17.   -15. ...    11.   543.   367.]
 ...
 [  390.   222.   228. ... -2764.    58.   -71.]
 [  383.   183.   177. ... -2740.    82.   -15.]
 [  275.    53.    94. ... -2840.   -63.  -182.]] [[ -298.   157.   152. ...  -208.   250.   128.]
 [  170.    34.    -8. ... -2844.    85.   210.]
 [ -382.   124.   114. ...    12.  

(1.0, 1.0)

In [50]:
experiment(ts4, ts6)

[[  81.   92.   86. ...  153.   71.   69.]
 [-339.   82.   71. ...   71.   32.   30.]
 [ 108.  116.  123. ...  178.   59.   91.]
 ...
 [ 168.  -50.  -70. ... -263.   -1.  -66.]
 [  77.   35.   29. ...   46.  -75.   67.]
 [ 209.  -21.  -72. ... -305.  -75. -135.]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[ -11.  -64.  -57. ...    0.  405.  266.]
 [-152.   43.  -19. ...  -35.  510.  328.]
 [  77.   35.   29. ...   46.  -75.   67.]
 ...
 [ 199.  141.   44. ... -153. -262.    2.]
 [  59.   65.   -2. ... -166. -151.  154.]
 [  15.    0.  -65. ...   85.   60.  232.]] [[  35.   62.   21. ...  128.   68.  173.]
 [ 159.  131.   70. ... -217. -233.   72.]
 [  81.   92.   86. ...  153.   71.   69.]
 ...
 [ 128.  114.   36. ... -233. -208.   91.]
 [  31.   33.  -32. ...  102.   76.  204.]
 [  71.   96.  107. ...  180.   91.  154.]] [1 1 0 1 0 1 1 0 0 0 0 1 1 1 1 0 1 1 1 0 1 0 0 0 1 0 1 1 0 1 1 1 0 1 1 1 0
 0 1 0 0 0

(1.0, 1.0)

In [51]:
experiment(ts5, ts6)

[[-323.  -55.  -79. ...  -93.  -55.  164.]
 [-339.   82.   71. ...   71.   32.   30.]
 [-265.   51.   29. ... -106. -126.  137.]
 ...
 [ 168.  -50.  -70. ... -263.   -1.  -66.]
 [  35.  -15.   66. ...  227.  197. -201.]
 [ 209.  -21.  -72. ... -305.  -75. -135.]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[  66.  -83.  -45. ...  -84.  261.  164.]
 [-370.  -12.   20. ...  -69. -233.   44.]
 [  38.  -73.  -43. ...  -40.  332.  199.]
 ...
 [ -97.  -36.  -76. ...   81.  530.  347.]
 [ 137.  -89.  -82. ... -178.  100.   31.]
 [-269. -138.   15. ...  303.  -17.  -51.]] [[-170.  104.   81. ... -112.  407.  265.]
 [-246.   68.   92. ...  -64. -266.  -22.]
 [-356.  177.  224. ... -196.  178.   78.]
 ...
 [-352.  -77. -135. ...  127. -209.  -78.]
 [-312. -215.  -64. ...  334.   13. -105.]
 [ -65.  -51.  -75. ...   59.  479.  323.]] [1 0 1 1 1 0 0 1 0 0 1 0 0 0 1 1 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 1
 0 1 1 1 0

(1.0, 1.0)

In [52]:
experiment(ts1, ts7)

[[ -36.  188.  324. ...  -18.  -71.  161.]
 [-546.  -98.   68. ...  130.  -61.   72.]
 [ -38.  192.  314. ...  -20.  -60.  186.]
 ...
 [  61.  186.  159. ...  -11.  -36.  205.]
 [ -19.   75.  -14. ... -106.   -1.  -31.]
 [  67.  170.  163. ...   48.   10.  236.]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[  10.  390.  344. ... -250. -380. -137.]
 [ -53.  187.  100. ...   -5. -229.  -48.]
 [-588.  -84.   44. ...   42. -113.   83.]
 ...
 [ -20.  196.  174. ...  -48.  -33.  209.]
 [-300.    5.  159. ...  -35.    4.  250.]
 [-126.  151.   92. ...  -62.  -28.  207.]] [[ -38.  192.  314. ...  -20.  -60.  186.]
 [ -84.  239.  269. ... -113. -121.   94.]
 [  47.  342.  310. ... -174. -413. -196.]
 ...
 [-669.   -6.   15. ...   63. -117.  135.]
 [-691.  -62.   51. ...  108.  -23.  271.]
 [-628.   -3.    3. ...   53. -112.  121.]] [0 0 1 1 1 1 1 1 0 1 1 1 0 0 0 0 1 1 0 0 1 0 1 1 0 0 1 1 0 0 0 1 0 0 0 0 1
 0 1 1 1 1

(1.0, 1.0)

In [53]:
experiment(ts2, ts7)

[[ -64. -143.   64. ...  -22.  108.  116.]
 [-546.  -98.   68. ...  130.  -61.   72.]
 [ -34. -100.   64. ...  -45.  122.  190.]
 ...
 [  61.  186.  159. ...  -11.  -36.  205.]
 [ 209.   20. -272. ... -250. -394. -264.]
 [  67.  170.  163. ...   48.   10.  236.]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[  67.  170.  163. ...   48.   10.  236.]
 [ -34. -100.   64. ...  -45.  122.  190.]
 [   9.   -5.  113. ...  -43.   66.  205.]
 ...
 [ -20.  196.  174. ...  -48.  -33.  209.]
 [-476.  -70.  -20. ...    6.   83.  314.]
 [ -59.  180.  162. ...  -49.  -27.  203.]] [[ -57.  -62.  -21. ...   78.   37.  269.]
 [ -25.  -52.   42. ...  -21.   27.  220.]
 [-235.   47.  168. ...  -46.   -4.  231.]
 ...
 [ -30.  -44.  122. ...  -77.   89.  196.]
 [-669.   -6.   15. ...   63. -117.  135.]
 [  61.  186.  159. ...  -11.  -36.  205.]] [1 0 0 1 1 1 1 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0 0 0 0 1 1 0 1
 0 0 1 1 1

(1.0, 1.0)

In [54]:
experiment(ts3, ts7)

[[  241.   177.   110. ... -3199.  -146.  -327.]
 [ -546.   -98.    68. ...   130.   -61.    72.]
 [  219.   148.    60. ... -3089.   -61.  -255.]
 ...
 [   61.   186.   159. ...   -11.   -36.   205.]
 [  169.    60.   -16. ... -2905.    67.   180.]
 [   67.   170.   163. ...    48.    10.   236.]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[   40.   227.   186. ...   -33.   -43.   205.]
 [  275.    53.    94. ... -2840.   -63.  -182.]
 [  143.    19.   -20. ... -2792.    78.  -134.]
 ...
 [  166.    46.   -33. ... -2900.    28.  -191.]
 [ -538.   -83.   -39. ...    30.    98.   328.]
 [ -235.    47.   168. ...   -46.    -4.   231.]] [[ -628.    -3.     3. ...    53.  -112.   121.]
 [ -708.   -66.    83. ...    82.   -70.   222.]
 [  149.     4.   -10. ... -2871.   -66.  -213.]
 ...
 [  225.    38.    25. ... -2852.   -66.  -187.]
 [  364.   174.   181. ... -2730.   102.    22.]
 [  173.    31.    41. ... 

(1.0, 1.0)

In [55]:
experiment(ts4, ts7)

[[  81.   92.   86. ...  153.   71.   69.]
 [-546.  -98.   68. ...  130.  -61.   72.]
 [ 108.  116.  123. ...  178.   59.   91.]
 ...
 [  61.  186.  159. ...  -11.  -36.  205.]
 [  77.   35.   29. ...   46.  -75.   67.]
 [  67.  170.  163. ...   48.   10.  236.]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[ 197.  124.   63. ... -157. -239.   32.]
 [ -17.    5.  -55. ... -127.  -49.  197.]
 [-669.   -6.   15. ...   63. -117.  135.]
 ...
 [-705.  -51.   82. ...   68.  -91.  185.]
 [ 104.  101.  145. ...  193.   70.   96.]
 [-126.  151.   92. ...  -62.  -28.  207.]] [[  87.   90.    7. ... -181. -175.  129.]
 [-476.  -70.  -20. ...    6.   83.  314.]
 [-300.    5.  159. ...  -35.    4.  250.]
 ...
 [-638.  -97.  -34. ...   82.   86.  349.]
 [-195.  103.  115. ...  -48.   -2.  224.]
 [-538.  -83.  -39. ...   30.   98.  328.]] [0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 1 1 1 0 0 0
 1 0 1 0 1

(1.0, 1.0)

In [56]:
experiment(ts5, ts7)

[[-323.  -55.  -79. ...  -93.  -55.  164.]
 [-546.  -98.   68. ...  130.  -61.   72.]
 [-265.   51.   29. ... -106. -126.  137.]
 ...
 [  61.  186.  159. ...  -11.  -36.  205.]
 [  35.  -15.   66. ...  227.  197. -201.]
 [  67.  170.  163. ...   48.   10.  236.]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[-269. -138.   15. ...  303.  -17.  -51.]
 [-349.  -15.  107. ...   -3.   35.  274.]
 [-320.  -12.   41. ...  -61. -238.   23.]
 ...
 [-315.  -12. -120. ...   88. -242.  -60.]
 [  64.   45.   77. ...   94.  -96. -415.]
 [  85.  109.  133. ...  137.  -97. -372.]] [[-538.  -83.  -39. ...   30.   98.  328.]
 [-235.   47.  168. ...  -46.   -4.  231.]
 [  85.  130.  151. ...  187.  -91. -322.]
 ...
 [-589. -103.   60. ...   91.  -85.   86.]
 [-379.  -27.  -89. ...    1. -269.  -57.]
 [  67.  224.  188. ...   -8.  -33.  188.]] [0 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1 0 0 1 0 0 0 1 1 1 1 0 1 0 0 0 1 0 1 0 1 0
 0 1 0 0 0

(1.0, 1.0)

In [57]:
experiment(ts6, ts7)

[[-339.   82.   71. ...   71.   32.   30.]
 [-546.  -98.   68. ...  130.  -61.   72.]
 [-393.   72.   59. ...   72.   95.   45.]
 ...
 [  61.  186.  159. ...  -11.  -36.  205.]
 [ 209.  -21.  -72. ... -305.  -75. -135.]
 [  67.  170.  163. ...   48.   10.  236.]] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[[-170.  104.   81. ... -112.  407.  265.]
 [-628.   -3.    3. ...   53. -112.  121.]
 [-300.    5.  159. ...  -35.    4.  250.]
 ...
 [ -59.  180.  162. ...  -49.  -27.  203.]
 [-235.   47.  168. ...  -46.   -4.  231.]
 [-361.  146.  140. ...  -61.  115.   71.]] [[-392.   97.  102. ...   68.  104.   69.]
 [-342.  196.  255. ... -152.  154.   48.]
 [-139.   17.  -15. ...   11.  543.  367.]
 ...
 [  92. -101.  -79. ... -117.  213.  135.]
 [ 168.  -50.  -70. ... -263.   -1.  -66.]
 [-588.  -84.   44. ...   42. -113.   83.]] [0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0 1 0 1 0 1 0 0 1 0 0
 0 1 1 1 0

(1.0, 1.0)

In [ ]:
print(ts1.shape)

In [ ]:
def make_experiment(ts1, ts2, len1=100, len2=100):
    t1 = ts1[:len1]
    t2 = ts2[:len2]
    #print(t1)
    #print(t2)
    #print(dtaidistance.dtw_ndim.warping_paths(t1, t2)[1])
    #print(dtaidistance.dtw_ndim.warping_path(t1, t2))
    return dtaidistance.dtw_ndim.warping_path(t1, t2)
    #return dtaidistance.dtw_ndim.distance(t1, t2)
    #dtaidistance.dtw_ndim.distance(t1, t2, window=None, max_dist=None, max_step=None, max_length_diff=None, penalty=None, psi=None, use_c=False)

In [90]:
make_experiment(ts1, ts2)

[[ -36.  188.  324. ...  326.  -38.   23.]
 [ -38.  192.  314. ...  318.   -6.   83.]
 [ -48.  193.  298. ...  310.    9.  133.]
 ...
 [  94.  199.  -72. ...  -28. -284. -166.]
 [  44.  172.  -96. ...   -4. -295. -135.]
 [ -11.  149. -123. ...  -18. -267.  -75.]]
[[ -64. -143.   64. ... -117. -183. -249.]
 [ -34. -100.   64. ... -108. -179. -258.]
 [ -30.  -44.  122. ...  -73. -151. -279.]
 ...
 [-264.  343.  184. ...  394. -316. -526.]
 [-279.  305.  117. ...  365. -337. -520.]
 [-296.  294.  116. ...  408. -323. -450.]]


AttributeError: module 'dtaidistance.dtw_ndim' has no attribute 'warping_path'